In [1]:
import glob

import os
from skimage import io
import csv
import pickle
import sys
import numpy as np

image_list = []
label_list = []
print("!!!")
for j in range(1,689):
    img_path = 'preview1/'+str(j) +"____" '*.jpeg'
    for infile in glob.glob(img_path):
        img = io.imread(infile)
#         img_normalized= XTrain = (img-np.min(img))/(np.max(img)-np.min(img))
        print(infile)
        image_list.append(img)
print("???")
with open("aug_labels_fin.csv", "r") as f:
    reader = csv.reader(f, delimiter="\t")
    for row in reader:
        vals = row[0].split(",")
        label_list.append(vals[1])


print("IMAGE LIST LENGTH")
print(len(image_list))
x_train_data=image_list[0:6994]

print("LABEL LIST LENGTH")
print(len(label_list))
y_train_data= label_list[0:6994]

!!!
preview1/1____0_3173.jpeg
preview1/1____0_9771.jpeg
preview1/1____0_1650.jpeg
preview1/1____0_7245.jpeg
preview1/1____0_7711.jpeg
preview1/1____0_3831.jpeg
preview1/1____0_4600.jpeg
preview1/1____0_2614.jpeg
preview1/1____0_1675.jpeg
preview1/1____0_4478.jpeg
preview1/1____0_4981.jpeg
preview1/1____0_7074.jpeg
preview1/1____0_266.jpeg
preview1/1____0_3884.jpeg
preview1/1____0_9524.jpeg
preview1/1____0_3399.jpeg
preview1/2____0_1139.jpeg
preview1/2____0_2832.jpeg
preview1/2____0_3302.jpeg
preview1/2____0_1854.jpeg
preview1/2____0_4915.jpeg
preview1/2____0_7346.jpeg
preview1/2____0_3926.jpeg
preview1/2____0_5552.jpeg
preview1/2____0_5164.jpeg
preview1/2____0_6629.jpeg
preview1/2____0_9819.jpeg
preview1/2____0_6383.jpeg
preview1/2____0_4128.jpeg
preview1/2____0_6399.jpeg
preview1/2____0_6994.jpeg
preview1/2____0_4887.jpeg
preview1/3____0_4350.jpeg
preview1/3____0_2866.jpeg
preview1/3____0_2797.jpeg
preview1/3____0_3836.jpeg
preview1/3____0_4335.jpeg
preview1/3____0_4616.jpeg
preview1/

KeyboardInterrupt: 

In [ ]:
file = open('data.pkl','rb')
display_data=pickle.load(file)
x_test_data=display_data["image"][-190:]
y_test_data=display_data["label"][-190:]
x_test_data=(x_test_data-np.min(x_test_data))/(np.max(x_test_data)-np.min(x_test_data))


In [ ]:
print(len(x_train_data),len(y_train_data))

In [ ]:
print(len(x_test_data),len(y_test_data))

In [ ]:
new_yTrain =[]
# tokenize the labels
for i in y_train_data:
    if(i == '-Fire-Smoke'):
        new_yTrain.append(0)
    elif (i == '-Fire+Smoke'):
        new_yTrain.append(1)
    elif (i == '+Fire+Smoke'):
        new_yTrain.append(2)
        
new_yTest =[]
# tokenize the labels
for i in y_test_data:
    if(i == '-Fire-Smoke'):
        new_yTest.append(0)
    elif (i == '-Fire+Smoke'):
        new_yTest.append(1)
    elif (i == '+Fire+Smoke'):
        new_yTest.append(2)


In [ ]:
from sklearn.model_selection import train_test_split
seed =12345
# XTrain, XTest, yTrain, yTest =x_train_data,x_test_data,new_yTrain,new_yTest
XTrain, XTest_f, yTrain, yTest_f = train_test_split(x_train_data,new_yTrain,test_size=0.001,random_state=seed)
# XTest,XTrain_f, yTest,yTrain_f = train_test_split(x_test_data,new_yTest,test_size=0.00,random_state=seed)
XTest=x_test_data
print(XTrain[0].shape)
XTrain = (XTrain-np.min(XTrain))/(np.max(XTrain)-np.min(XTrain))
print(XTrain[0].shape)
valid_data = []
valid_data.append(XTest)

In [ ]:
from keras.utils import np_utils
yTest = new_yTest
yTrain = np_utils.to_categorical(yTrain)
yTest = np_utils.to_categorical(yTest)
print(yTrain[0],yTest[0])
valid_data.append(yTest)

In [ ]:
from keras.applications import VGG16
from keras import models
from keras import layers
from keras import optimizers

#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(300, 600, 3))


    
# Freeze the last 3 layers
print("Freeze the last 3 layers")
for layer in vgg_conv.layers[:-3]:
    layer.trainable = False

# Create the model
model = models.Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(4096, activation='relu'))
model.add(layers.Dense(1000, activation='relu'))
model.add(layers.Dropout(0.8))
model.add(layers.Dense(3, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model

history=model.fit(XTrain, yTrain, epochs=10, batch_size=25,validation_data=valid_data)

print(history.history.keys())
# Save the model
fileName="freeze_last3_FP_augmented_data_001_512_128_vgg_imagenet.h5"
model.save(fileName)

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))



In [ ]:
import matplotlib.pyplot as plt
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()